# BDD hybride par ChromaDB

# Explications

Les bases de données vectorielles hybrides, comme **ChromaDB**, combinent **des vecteurs** (embeddings) pour la recherche sémantique avec **des données classiques** (métadonnées) pour le filtrage et l'organisation. Contrairement aux bases de données classiques (SQL), elles sont optimisées pour les requêtes **approximate nearest neighbor (ANN)** et les recherches **hybrides**.

---

## Structure des collections

- 🔹 **Collections vs Tables** :
  - En **SQL**, on utilise des **tables** avec des colonnes et des types fixes.
  - En **ChromaDB**, on utilise des **collections**, qui stockent des **vecteurs**, des **IDs**, et des **métadonnées JSON**.
  
- 🔹 **Données stockées** dans chaque collection :
  - **Embeddings** : représentations numériques (vecteurs) des données pour la recherche sémantique.
  - **IDs uniques** : identifiants pour récupérer des éléments.
  - **Métadonnées** : informations complémentaires sous forme de dictionnaire JSON.

- 🔹 **Absence de schéma rigide** :
  - Contrairement aux bases SQL, **les collections sont flexibles** : pas besoin de définir à l'avance les colonnes ou types de données.
  - Les **métadonnées peuvent varier d’un élément à l’autre**.

---

## Lien entre les collections

- 🔗 **Pas de relations directes comme en SQL** (pas de `JOIN`) → On simule les relations avec des **IDs croisés ("foreign keys")**.

- 📌 **Stratégies pour gérer les relations** :
  - Ajouter un **champ ID de référence** dans les métadonnées (ex: `"author_id": "auteur_1"` pour lier une image à un auteur).
  - Effectuer des **requêtes en deux étapes** :
    1. Rechercher un élément dans une collection.
    2. Utiliser son ID pour récupérer les données associées dans une autre collection.

  - Exemple d'association **1-N** (un auteur a plusieurs images) :

    ```python
    # Récupérer toutes les images d'un auteur donné
    collection.get(where={"author_id": "auteur_1"})
    ```

  - 📍 Contrairement à SQL, pas de **contrainte d’intégrité référentielle** → Il faut gérer manuellement la suppression et la mise à jour des liens.

---

## Métadonnées

- 🏷️ **Stockées en JSON**, flexibles, et utilisées pour le **filtrage rapide**.
- 🔍 **Utilisées pour des recherches hybrides** :
  - **Recherche vectorielle** pour la similarité sémantique.
  - **Filtrage classique** sur des attributs spécifiques.
  
- 📊 **Avantages des métadonnées** :
  - Permettent d’ajouter des **catégories, tags, sources, relations** aux données vectorielles.
  - Accélèrent la recherche en limitant l’espace de recherche (ex: **ne chercher que dans une catégorie donnée**).

- ✅ **Exemple de requête hybride** (trouver les 3 images les plus proches d’un vecteur, mais uniquement dans la catégorie "animal") :

  ```python
  collection.query(
      query_embeddings=[[0.1, 0.2, 0.3]],
      n_results=3,
      where={"category": "animal"}
  )


## regroupement de toutes les variables en 1 table

In [45]:
import chromadb

In [46]:
# Création du client et de la base locale
client = chromadb.PersistentClient(path="./chroma_db")

In [ ]:
# Création d'une collection
detection = client.get_or_create_collection("detection")
Image = client.get_or_create_collection("image")

### Fonction d'ajout dans la collection

In [48]:
# TODO : ajout de catégories obligatoires pour les collections

# Ajout de documents avec vecteurs + métadonnées
def add_detection(ids, embeddings, metadatas):
    detection.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=metadatas
    )

# REFLEXION : on a une ligne par objet détecté, alors qu'il faudrait une ligne par frame 


### Génération aléatoires de données

In [49]:
# génération aléatoire de données
import random

ids = [str(i) for i in range(10)]
embeddings = [[random.random() for _ in range(3)] for _ in range(10)]
metadatas = [{
    "nom_image": f"frame{i:03d}.jpg",
    "num_frame": str(i),
    "nom_video": f"video{i//10:03d}.mp4",
    "nb_frames_tot": "150",
    "fps": "15",
    "resolution": "1920x1080",
    "class_name": random.choice(["person", "car", "animal"]),
    "x": str(random.randint(0, 1920)),
    "y": str(random.randint(0, 1080)),
    "height": str(random.randint(100, 500)),
    "width": str(random.randint(100, 500))
} for i in range(10)]

add_detection(ids, embeddings, metadatas)

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9


### Affichage de toutes les données

In [50]:
# Récupérer toutes les données (ids, embeddings, metadatas)
all_data = detection.get(include=["embeddings", "metadatas"])

# Afficher tout le contenu
for i in range(len(all_data["ids"])):
    print(f"🔹 ID: {all_data['ids'][i]}")
    if all_data['embeddings'] is not None:
        print(f"🧠 Embedding: {all_data['embeddings'][i]}")
    print(f"📌 Métadonnées: {all_data['metadatas'][i]}")
    print("-" * 40)


🔹 ID: 1
🧠 Embedding: [0.1        0.2        0.30000001]
📌 Métadonnées: {'class_name': 'person', 'fps': '15', 'height': '400', 'nb_frames_tot': '150', 'nom_image': 'frame001.jpg', 'nom_video': 'video001.mp4', 'num_frame': '1', 'resolution': '1920x1080', 'width': '100', 'x': '400', 'y': '200'}
----------------------------------------
🔹 ID: 2
🧠 Embedding: [0.1        0.2        0.30000001]
📌 Métadonnées: {'class_name': 'person', 'fps': '15', 'height': '400', 'nb_frames_tot': '150', 'nom_image': 'frame001.jpg', 'nom_video': 'video001.mp4', 'num_frame': '1', 'resolution': '1920x1080', 'width': '100', 'x': '400', 'y': '200'}
----------------------------------------
🔹 ID: 4
🧠 Embedding: [0.1        0.2        0.30000001]
📌 Métadonnées: {'class_name': 'person', 'fps': '15', 'height': '400', 'nb_frames_tot': '150', 'nom_image': 'frame001.jpg', 'nom_video': 'video001.mp4', 'num_frame': '1', 'resolution': '1920x1080', 'width': '100', 'x': '400', 'y': '200'}
---------------------------------------

### Query

In [41]:
def ask_query(collection, query_vector=[0], query={"":""}, max_dist=0.5):
    # Obtenir le nombre de documents dans la collection
    num_documents = len(collection.get()["ids"])

    # Recherche hybride (similitude vectorielle + filtre sur la catégorie)
    results = detection.query(
        query_embeddings=[query_vector],
        n_results=num_documents,  # Nombre de résultats souhaités
        where=query,  # Filtrage par métadonnée
        include=["embeddings", "metadatas", "distances"],
    )

    filtered_results = {    # Filtrer les résultats pour n'afficher que ceux avec une distance inférieure à 0.5
            "ids": [],
            "embeddings": [],
            "metadatas": [],
            "distances": []
        }

    for i in range(len(results["ids"])):
        if results["distances"][i][0] < max_dist:  # Access the first element of the list
            filtered_results["ids"].append(results["ids"][i])
            filtered_results["embeddings"].append(results["embeddings"][i])
            filtered_results["metadatas"].append(results["metadatas"][i])
            filtered_results["distances"].append(results["distances"][i][0])  # Access the first element of the list

    results = filtered_results

    return results


In [56]:
results = ask_query(detection, query_vector=[0.9, 0.4, 0.3], query={"height":"400"}, max_dist=0.7)

# Affichage des résultats de la recherche
for i in range(len(results["ids"])):
    print(f"🔹 ID: {results['ids'][i]}")
    print(f"🧠 Embedding: {results['embeddings'][i]}")
    print(f"📌 Métadonnées: {results['metadatas'][i]}")
    print(f"📏 Distance: {results['distances'][i]}")
    print("-" * 40)

🔹 ID: ['yippi', '4', '2', '1']
🧠 Embedding: [[0.1        0.2        0.30000001]
 [0.1        0.2        0.30000001]
 [0.1        0.2        0.30000001]
 [0.1        0.2        0.30000001]]
📌 Métadonnées: [{'class_name': 'person', 'fps': '15', 'height': '400', 'nb_frames_tot': '150', 'nom_image': 'frame001.jpg', 'nom_video': 'video001.mp4', 'num_frame': '1', 'resolution': '1920x1080', 'width': '100', 'x': '400', 'y': '200'}, {'class_name': 'person', 'fps': '15', 'height': '400', 'nb_frames_tot': '150', 'nom_image': 'frame001.jpg', 'nom_video': 'video001.mp4', 'num_frame': '1', 'resolution': '1920x1080', 'width': '100', 'x': '400', 'y': '200'}, {'class_name': 'person', 'fps': '15', 'height': '400', 'nb_frames_tot': '150', 'nom_image': 'frame001.jpg', 'nom_video': 'video001.mp4', 'num_frame': '1', 'resolution': '1920x1080', 'width': '100', 'x': '400', 'y': '200'}, {'class_name': 'person', 'fps': '15', 'height': '400', 'nb_frames_tot': '150', 'nom_image': 'frame001.jpg', 'nom_video': 'vide

#